In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [16]:
url = "https://www.mercadolivre.com.br/ofertas?container_id=MLB779362-1&page="

data = requests.get(url).text

In [8]:
type(data)

str

In [19]:
with open('mercadolivre.html', 'w', encoding='utf-8') as f:
          f.write(data)

doc = BeautifulSoup(data, 'html.parser')

In [10]:
# retorna titulos dos produtos nas tags
def get_itens_title(doc):
    title_tags = doc.find_all('p', class_ = 'promotion-item__title')
    titles = []
    for tags in title_tags:
        titles.append(tags.text)
    
    return titles

In [20]:
# retorna precos dos produtos
def get_itens_price(doc):
    price_tags = doc.find_all('div', class_ = 'andes-money-amount-combo__main-container')
    price = []
    for tags in price_tags:
        price.append(tags.text.replace('Â', ''))
    
    return price

In [21]:
def get_doc(url):
    response = requests.get(url)
    doc = BeautifulSoup(response.text, 'html.parser')
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(response))
    return doc

In [22]:
def scrape_multiple_pages(n):
    titles, prices = [], []

    for page in range (1, n+1):
        doc = get_doc(url + str(page))
        titles.extend(get_itens_title(doc))
        prices.extend(get_itens_price(doc))
    
    itens = {
        'TITLE': titles,
        'PRICE': prices
    }

    return pd.DataFrame(itens)

In [26]:
df_mercado_livre = scrape_multiple_pages(10)
df_mercado_livre["SCRAPY_DATETIME"] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
df_mercado_livre['PRICE'] = df_mercado_livre['PRICE'].str.extract(r'(\d+[\.,]?\d*)')
df_mercado_livre

,TITLE,PRICE,SCRAPY_DATETIME
0,Samsung Galaxy A14 (Exynos) 5G Dual SIM 128 GB...,799,2023-11-25 15:19:17
1,Smart TV Philco Ptv32m8gagcmblh LED Android TV...,859,2023-11-25 15:19:17
2,Samsung Galaxy Tab S6 Lite 128gb 4gb Ram Cinza,1599,2023-11-25 15:19:17
3,Robô Aspirador Wap Robot W90 3 Modos De Limpez...,294,2023-11-25 15:19:17
4,Playstation 5 825gb Disco + Bundle Ea Sports F...,3999,2023-11-25 15:19:17
...,...,...,...
475,Aspirador extrator é industrial Tambor Wap Car...,1499,2023-11-25 15:19:17
476,"Tablet Multilaser Kid Pad NB380 7"" 32GB laran...",299,2023-11-25 15:19:17
477,Samsung Galaxy A54 5g 256gb 8gb Ram Violeta,1799,2023-11-25 15:19:17
478,Tablet Multilaser Patrulha Canina Skye Wi-fi 1...,369,2023-11-25 15:19:17
